In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.optim import AdamW, LBFGS
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np
import os
import pandas as pd
from itertools import cycle

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
def fourier_features(x, B):
    x_transformed = torch.matmul(x, B)
    return torch.cat([torch.sin(x_transformed), torch.cos(x_transformed)], dim=-1)

def init_fixed_frequency_matrix(size, scale=1.0):
    num_elements = size[0] * size[1]
    lin_space = torch.linspace(-scale, scale, steps=num_elements)
    B = lin_space.view(size).float()
    return B

class FourierFeatureNN(nn.Module):
    def __init__(self, input_dim=1, shared_units=16, neuron_units=32, scale=1.0, 
                 activation=nn.Tanh, device='cpu'):
        super(FourierFeatureNN, self).__init__()
        self.Bx = init_fixed_frequency_matrix((input_dim, shared_units // 2), scale=scale).to(device)
        self.Bt = init_fixed_frequency_matrix((input_dim, shared_units // 2), scale=scale).to(device)

        # Define separate paths for x and t after Fourier transformation
        self.path_x = nn.Sequential( 
            nn.Linear(shared_units, neuron_units),  # Adjusted from shared_units // 2 to shared_units
            activation(),
            nn.Linear(neuron_units, neuron_units),
            activation(),
            nn.Linear(neuron_units, neuron_units),
            activation() )
        self.path_t = nn.Sequential( 
            nn.Linear(shared_units, neuron_units),  # Same adjustment
            activation(),
            nn.Linear(neuron_units, neuron_units),
            activation(),
            nn.Linear(neuron_units, neuron_units),
            activation() )

        # Define separate FFN for u and v directly after the paths
        self.ffn_u = nn.Sequential(
            nn.Linear(neuron_units, neuron_units), activation(),
            nn.Linear(neuron_units, neuron_units), activation(),
            nn.Linear(neuron_units, 2)  # Outputs for u (real and imaginary parts)
        )
        

        self.apply(self.initialize_weights)

    def forward(self, x, t):
        # Apply Fourier feature transformations
        x_fourier = fourier_features(x, self.Bx)
        t_fourier = fourier_features(t, self.Bt)

        # Pass through separate paths
        x_path_output = self.path_x(x_fourier)
        t_path_output = self.path_t(t_fourier)

        # Pointwise multiplication of the separate path outputs
        combined_features = x_path_output * t_path_output

        # Directly pass through different FFNs for u and v
        final_output_u = self.ffn_u(combined_features)
        #final_output_v = self.ffn_v(combined_features)

        # Splitting the output for u into real and complex parts
        output_1, output_2 = final_output_u.split(1, dim=-1)
        #output_3 = final_output_v
        
        return output_1, output_2#, output_3

    def initialize_weights(self, m):
        if isinstance(m, nn.Linear):
            init.xavier_uniform_(m.weight)
            if m.bias is not None:
                init.constant_(m.bias, 0)

In [3]:
def fourier_features(x, B):
    x_transformed = torch.matmul(x, B)
    return torch.cat([torch.sin(x_transformed), torch.cos(x_transformed)], dim=-1)

def init_fixed_frequency_matrix(size, scale=1.0):
    num_elements = size[0] * size[1]
    lin_space = torch.linspace(-scale, scale, steps=num_elements)
    B = lin_space.view(size).float()
    return B

class FourierFeatureNN_V(nn.Module):
    def __init__(self, input_dim=1, shared_units=16, neuron_units=32, scale=1.0, 
                 activation=nn.Tanh, device='cpu'):
        super(FourierFeatureNN_V, self).__init__()
        self.Bx = init_fixed_frequency_matrix((input_dim, shared_units // 2), scale=scale).to(device)
        self.Bt = init_fixed_frequency_matrix((input_dim, shared_units // 2), scale=scale).to(device)

        # Define separate paths for x and t after Fourier transformation
        self.path_x = nn.Sequential( 
            nn.Linear(shared_units, neuron_units),  # Adjusted from shared_units // 2 to shared_units
            activation(),
            nn.Linear(neuron_units, neuron_units),
            activation(),
            nn.Linear(neuron_units, neuron_units),
            activation() )
        self.path_t = nn.Sequential( 
            nn.Linear(shared_units, neuron_units),  # Same adjustment
            activation(),
            nn.Linear(neuron_units, neuron_units),
            activation(),
            nn.Linear(neuron_units, neuron_units),
            activation() )

        # Define separate FFN for u and v directly after the paths
        self.ffn_v = nn.Sequential(
            nn.Linear(neuron_units, neuron_units), activation(),
            nn.Linear(neuron_units, neuron_units), activation(),
            nn.Linear(neuron_units, 1)  # Outputs for u (real and imaginary parts)
        )
        

        self.apply(self.initialize_weights)

    def forward(self, x, t):
        # Apply Fourier feature transformations
        x_fourier = fourier_features(x, self.Bx)
        t_fourier = fourier_features(t, self.Bt)

        # Pass through separate paths
        x_path_output = self.path_x(x_fourier)
        t_path_output = self.path_t(t_fourier)

        # Pointwise multiplication of the separate path outputs
        combined_features = x_path_output * t_path_output

        # Directly pass through different FFNs for u and v
        final_output_v = self.ffn_v(combined_features)
        #final_output_v = self.ffn_v(combined_features)

        # Splitting the output for u into real and complex parts
       # output_1, output_2 = final_output_u.split(1, dim=-1)
        #output_3 = final_output_v
        
        return final_output_v

    def initialize_weights(self, m):
        if isinstance(m, nn.Linear):
            init.xavier_uniform_(m.weight)
            if m.bias is not None:
                init.constant_(m.bias, 0)

In [4]:
class FCN(nn.Module):
    def __init__(self, input_dim=1, neuron_units=32, activation=nn.Tanh, device='cpu'):
        super(FCN, self).__init__()

        # Define a simple FFN with 3 hidden layers
        self.ffn = nn.Sequential(
            nn.Linear(2 * input_dim, neuron_units),  # input dimension is doubled for x and t
            activation(),
            nn.Linear(neuron_units, neuron_units),
            activation(),
            nn.Linear(neuron_units, neuron_units),
            activation(),
            nn.Linear(neuron_units, 1)  # Outputs for u (2 parts: real and imaginary) and v (1 part)
        )

        self.to(device)
        self.apply(self.initialize_weights)

    def forward(self, x, t):
        # Concatenate x and t as input
        combined_input = torch.cat([x, t], dim=-1)
        output = self.ffn(combined_input)
        
        return output

    def initialize_weights(self, m):
        if isinstance(m, nn.Linear):
            init.xavier_uniform_(m.weight)
            if m.bias is not None:
                init.constant_(m.bias, 0)

In [5]:
def real_u1(x, t, k, omega, r):
    u = torch.sqrt(torch.tensor(2)) *( 1/torch.cosh(x+t) ) * torch.exp(1j * ( -x*torch.tensor(0.5) + t*torch.tensor(0.75) ) )
    return torch.real(u)

def imag_u1(x, t, k, omega, r):
    u = torch.sqrt(torch.tensor(2)) *( 1/torch.cosh(x+t) ) * torch.exp(1j * ( -x*torch.tensor(0.5) + t*torch.tensor(0.75) ) )
    return torch.imag(u)

def real_v1(x, t, k, omega, r):
    result = -2 * torch.square( 1/torch.cosh(x+t) )
    return result

def compute_analytical_boundary_loss(model_u, model_v, x, t, mse_cost_function, k, omega, r):
    pred_u_r, pred_u_i = model_u(x,t)
    pred_v =  model_v(x,t)

    real_u1_val = real_u1(x, t, k, omega, r)
    imag_u1_val = imag_u1(x, t, k, omega, r)
    real_v1_val = real_v1(x, t, k, omega, r)
 
    boundary_loss_ur = mse_cost_function(pred_u_r, real_u1_val)
    boundary_loss_ui = mse_cost_function(pred_u_i, imag_u1_val)
    boundary_loss_v = mse_cost_function(pred_v, real_v1_val)
    
    return boundary_loss_ur, boundary_loss_ui, boundary_loss_v

def cyclic_iterator(items):
    return cycle(items)

In [6]:
def LBFGS_training(model_u, model_v, model_save_path, mse_cost_function, device, num_epochs, lr, num_samples, r, k, omega, gamma, beta, line_search_fn):
    print('Starting LBFGS Fine Tuning')
    optimizer_u = LBFGS(model_u.parameters(), lr=lr, max_iter=20, max_eval=None, tolerance_grad=1e-07, tolerance_change=1e-09, history_size=100, line_search_fn=line_search_fn)
    #optimizer_ui = LBFGS(model_ui.parameters(), lr=lr, max_iter=20, max_eval=None, tolerance_grad=1e-07, tolerance_change=1e-09, history_size=100, line_search_fn=line_search_fn)
    optimizer_v = LBFGS(model_v.parameters(), lr=lr, max_iter=20, max_eval=None, tolerance_grad=1e-07, tolerance_change=1e-09, history_size=100, line_search_fn=line_search_fn)

    factor = -2

    x_n = (torch.rand(num_samples, 1)*4 + factor ).to(device)  # x in range [-5, -3]
    t_n = (torch.rand(num_samples, 1)).to(device)   
    x_dom = (torch.rand(num_samples*30, 1)*4 + factor ).to(device)
    t_dom = torch.rand(num_samples*30, 1).to(device) 
    x_bc_x0 = (torch.zeros(num_samples, 1)*4 + factor ).to(device)
    t_bc_x0 = torch.rand(num_samples, 1).to(device)  # Uniformly distributed random values between 0 and 1
    x_bc_x1 = (torch.zeros(num_samples, 1)*4 - factor ).to(device)
    t_bc_x1 = torch.rand(num_samples, 1).to(device)  # Uniformly distributed random values between 0 and 1
    x_bc_t0 = (torch.rand(num_samples, 1)*4 + factor ).to(device)  # Uniformly distributed random values between 0 and 1
    t_bc_t0 = torch.zeros(num_samples, 1).to(device)

    for epoch in tqdm(range(num_epochs),
                  desc='Progress:',  
                  leave=False,  
                  ncols=75,
                  mininterval=0.1,
                  bar_format='{l_bar} {bar} | {remaining}',  # Only show the bar without any counters
                  colour='blue'): 
        model_u.train()
        model_v.train()
        
        def closure_u():
            optimizer_u.zero_grad()
            optimizer_v.zero_grad()
            x_dom = (torch.rand(num_samples*30, 1)*4 + factor ).to(device)
            t_dom = torch.rand(num_samples*30, 1).to(device) 
            x_dom.requires_grad_(True)
            t_dom.requires_grad_(True)

            #physics_loss_ur, physics_loss_ui, physics_loss_v = compute_physics_loss(model, x_n, t_n, device, mse_cost_function) 
            u_real, u_imag = model_u(x_dom, t_dom)
            v = model_v(x_dom, t_dom)
            u_abs = torch.square(u_real) + torch.square(u_imag)

            # First order derivatives with retain_graph=True to reuse computational graph
            u_real_x = torch.autograd.grad(u_real.sum(), x_dom, create_graph=True )[0]
            u_real_t = torch.autograd.grad(u_real.sum(), t_dom, create_graph=True )[0]
            u_imag_x = torch.autograd.grad(u_imag.sum(), x_dom, create_graph=True )[0]
            u_imag_t = torch.autograd.grad(u_imag.sum(), t_dom, create_graph=True )[0]
            v_x = torch.autograd.grad(v.sum(), x_dom, create_graph=True )[0]
            v_t = torch.autograd.grad(v.sum(), t_dom, create_graph=True )[0]
    
            # Second order derivatives
            u_real_xx = torch.autograd.grad(u_real_x.sum(), x_dom, create_graph=True )[0]
            u_imag_xx = torch.autograd.grad(u_imag_x.sum(), x_dom, create_graph=True )[0]

            # Compute u_abs_xx with retain_graph if further gradients need to be calculated
            u_abs_x = torch.autograd.grad(u_abs.sum(), x_dom, create_graph=True )[0]

            # Define du_r, du_i, dv according to given formulas
            du_r = - u_imag_t - u_real_xx - u_real * v
            du_i =   u_real_t - u_imag_xx - u_imag * v
            #dv = v_t + u_abs_x

            zero_target = torch.zeros_like(du_r)  # Assuming du_r, du_i, dv have the same shape
            physics_loss_ur = mse_cost_function(du_r, zero_target)
            physics_loss_ui = mse_cost_function(du_i, zero_target)
            #physics_loss_v = mse_cost_function(dv, zero_target)
            print(physics_loss_ur)
            print(physics_loss_ur)
            #print(physics_loss_v)

            boundary_loss_ur_x0, boundary_loss_ui_x0, boundary_loss_v_x0 = compute_analytical_boundary_loss(model_u, model_v, x_bc_x0, t_bc_x0, mse_cost_function, k, omega, r)
            boundary_loss_ur_x1, boundary_loss_ui_x1, boundary_loss_v_x1 = compute_analytical_boundary_loss(model_u, model_v, x_bc_x1, t_bc_x1, mse_cost_function, k, omega, r)
            boundary_loss_ur_t0, boundary_loss_ui_t0, boundary_loss_v_t0 = compute_analytical_boundary_loss(model_u, model_v, x_bc_t0, t_bc_t0, mse_cost_function, k, omega, r)
            #boundary_loss_ur_t1, boundary_loss_ui_t1, boundary_loss_v_t1 = compute_analytical_boundary_loss(model, x_bc_t1, t_bc_t1, mse_cost_function, k, omega, r)
            #domain_loss_ur_t, domain_loss_ui_t, domain_loss_v_t = compute_analytical_boundary_loss(model, x_dom, t_dom, mse_cost_function, k, omega, r)
            
            # Total loss 
            loss_ur = gamma*(physics_loss_ur) + beta*( boundary_loss_ur_x0 + boundary_loss_ur_x1 + boundary_loss_ur_t0)
            loss_ui = gamma*(physics_loss_ui) + beta*( boundary_loss_ui_x0 + boundary_loss_ui_x1 + boundary_loss_ur_t0 )
            #loss_v = gamma*(physics_loss_v) + beta*( boundary_loss_v_x0 + boundary_loss_v_x1 + boundary_loss_v_t0 )
            total_loss = loss_ur + loss_ui #+ loss_v
            total_loss.backward()

            return total_loss 


        def closure_v():
            optimizer_u.zero_grad()
            optimizer_v.zero_grad()
            x_dom = (torch.rand(num_samples*30, 1)*4 + factor ).to(device)
            t_dom = torch.rand(num_samples*30, 1).to(device) 
            x_dom.requires_grad_(True)
            t_dom.requires_grad_(True)

            #physics_loss_ur, physics_loss_ui, physics_loss_v = compute_physics_loss(model, x_n, t_n, device, mse_cost_function) 
            u_real, u_imag = model_u(x_dom, t_dom)
            v = model_v(x_dom, t_dom)
            u_abs = torch.square(u_real) + torch.square(u_imag)

            # First order derivatives with retain_graph=True to reuse computational graph
            u_real_x = torch.autograd.grad(u_real.sum(), x_dom, create_graph=True )[0]
            u_real_t = torch.autograd.grad(u_real.sum(), t_dom, create_graph=True )[0]
            u_imag_x = torch.autograd.grad(u_imag.sum(), x_dom, create_graph=True )[0]
            u_imag_t = torch.autograd.grad(u_imag.sum(), t_dom, create_graph=True )[0]
            v_x = torch.autograd.grad(v.sum(), x_dom, create_graph=True )[0]
            v_t = torch.autograd.grad(v.sum(), t_dom, create_graph=True )[0]
    
            # Second order derivatives
            u_real_xx = torch.autograd.grad(u_real_x.sum(), x_dom, create_graph=True )[0]
            u_imag_xx = torch.autograd.grad(u_imag_x.sum(), x_dom, create_graph=True )[0]

            # Compute u_abs_xx with retain_graph if further gradients need to be calculated
            u_abs_x = torch.autograd.grad(u_abs.sum(), x_dom, create_graph=True )[0]

            # Define du_r, du_i, dv according to given formulas
            #du_r = - u_imag_t - u_real_xx - u_real * v
            #du_i =   u_real_t - u_imag_xx - u_imag * v
            dv = v_t + u_abs_x

            zero_target = torch.zeros_like(dv)  # Assuming du_r, du_i, dv have the same shape
            #physics_loss_ur = mse_cost_function(du_r, zero_target)
            #physics_loss_ui = mse_cost_function(du_i, zero_target)
            physics_loss_v = mse_cost_function(dv, zero_target)
            #print(physics_loss_ur)
            #print(physics_loss_ur)
            print(physics_loss_v)

            boundary_loss_ur_x0, boundary_loss_ui_x0, boundary_loss_v_x0 = compute_analytical_boundary_loss(model_u, model_v, x_bc_x0, t_bc_x0, mse_cost_function, k, omega, r)
            boundary_loss_ur_x1, boundary_loss_ui_x1, boundary_loss_v_x1 = compute_analytical_boundary_loss(model_u, model_v, x_bc_x1, t_bc_x1, mse_cost_function, k, omega, r)
            boundary_loss_ur_t0, boundary_loss_ui_t0, boundary_loss_v_t0 = compute_analytical_boundary_loss(model_u, model_v, x_bc_t0, t_bc_t0, mse_cost_function, k, omega, r)
            #boundary_loss_ur_t1, boundary_loss_ui_t1, boundary_loss_v_t1 = compute_analytical_boundary_loss(model, x_bc_t1, t_bc_t1, mse_cost_function, k, omega, r)
            #domain_loss_ur_t, domain_loss_ui_t, domain_loss_v_t = compute_analytical_boundary_loss(model, x_dom, t_dom, mse_cost_function, k, omega, r)
            
            # Total loss 
            #loss_ur = gamma*(physics_loss_ur) + beta*( boundary_loss_ur_x0 + boundary_loss_ur_x1 + boundary_loss_ur_t0)
            #loss_ui = gamma*(physics_loss_ui) + beta*( boundary_loss_ui_x0 + boundary_loss_ui_x1 + boundary_loss_ur_t0 )
            loss_v = gamma*(physics_loss_v) + beta*( boundary_loss_v_x0 + boundary_loss_v_x1 + boundary_loss_v_t0 )
            total_loss = loss_v
            total_loss.backward()

            return total_loss 
    
        optimizer_u.step(closure_u)    
        optimizer_v.step(closure_v)

        
        
        if epoch % 10 == 0:
            current_loss_u = closure_u()  # Optionally recompute to print
            current_loss_v = closure_v()
            print(f' Epoch {epoch}, Loss U: {current_loss_u.item()}, Loss V: {current_loss_v.item()}') 
            model_u_filename = os.path.join(model_save_path, f'C_HIGGS_U_second_training_epoch_{epoch}.pth')
            torch.save(model_u.state_dict(), model_u_filename)
            model_v_filename = os.path.join(model_save_path, f'C_HIGGS_V_second_training_epoch_{epoch}.pth')
            torch.save(model_v.state_dict(), model_v_filename)
            plot_model_results(epoch, model_u, model_v, device, k, omega, r, sigma=1, cmap='viridis', image_save_path='results') 
            
    model_u_filename = os.path.join(model_save_path, f'C_HIGGS_U_second_training.pth')
    torch.save(model_u.state_dict(), model_u_filename)
    model_v_filename = os.path.join(model_save_path, f'C_HIGGS_V_second_training.pth')
    torch.save(model_v.state_dict(), model_v_filename)
    print('TRAINING COMPLETED')

In [7]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.ndimage import gaussian_filter

def plot_model_results(epoch, model_u, model_v, device, k, omega, r, sigma=1, cmap='viridis', image_save_path='results'):
    x = torch.linspace(-1.8, 1.8, 400)
    t = torch.linspace(0.2, 0.8, 400)
    X, T = torch.meshgrid(x, t)  # Create a 2D grid of x and t
    X_flat = X.flatten().unsqueeze(-1).to(device)
    T_flat = T.flatten().unsqueeze(-1).to(device)
    
    model_u_state = torch.load(os.path.join(model_save_path, f'C_HIGGS_U_second_training_epoch_{epoch}.pth'), map_location=device)
    model_u.load_state_dict(model_u_state)
    model_u.eval()

    model_v_state = torch.load(os.path.join(model_save_path, f'C_HIGGS_V_second_training_epoch_{epoch}.pth'), map_location=device)
    model_v.load_state_dict(model_v_state)
    model_v.eval()

    # Get predictions from the trained models
    with torch.no_grad():
        pred_u_r, pred_u_i = model_u(X_flat, T_flat) 
        pred_v = model_v(X_flat, T_flat) 

    pred_u_r = pred_u_r.cpu().reshape(X.shape).numpy()
    pred_u_i = pred_u_i.cpu().reshape(X.shape).numpy()
    pred_v = pred_v.cpu().reshape(X.shape).numpy()

    real_u1_analytical = real_u1(X_flat, T_flat, k, omega, r).cpu().reshape(X.shape).numpy()
    imag_u1_analytical = imag_u1(X_flat, T_flat, k, omega, r).cpu().reshape(X.shape).numpy()
    real_v1_analytical = real_v1(X_flat, T_flat, k, omega, r).cpu().reshape(X.shape).numpy()

    pred_v_smooth = gaussian_filter(pred_v, sigma=sigma)

    shrink = 0.3
    aspect = 50

    # Plotting predictions
    fig = plt.figure(figsize=(24, 16))

    ax1 = fig.add_subplot(231, projection='3d')
    ax1.plot_surface(X.numpy(), T.numpy(), pred_u_r, cmap=cmap)
    ax1.set_title('Predicted Real Part of $u_1(x, t)$')
    ax1.set_xlabel('x')
    ax1.set_ylabel('t')
    ax1.set_zlabel('Real part of $u_1$')

    ax2 = fig.add_subplot(232, projection='3d')

    ax2.plot_surface(X.numpy(), T.numpy(), pred_u_i, cmap=cmap)
    ax2.set_title('Predicted Imaginary Part of $u_1(x, t)$')
    ax2.set_xlabel('x')
    ax2.set_ylabel('t')
    ax2.set_zlabel('Imag part of $u_1$')

    ax3 = fig.add_subplot(233, projection='3d')
    ax3.plot_surface(X.numpy(), T.numpy(), pred_v_smooth, cmap=cmap)
    ax3.set_title('Predicted Real Part of $v_1(x, t)$')
    ax3.set_xlabel('x')
    ax3.set_ylabel('t')
    ax3.set_zlabel('Real part of $v_1$')

    ax4 = fig.add_subplot(234, projection='3d')
    ax4.plot_surface(X.numpy(), T.numpy(), real_u1_analytical, cmap=cmap)
    ax4.set_title('Analytical Real Part of $u_1(x, t)$')
    ax4.set_xlabel('x')
    ax4.set_ylabel('t')
    ax4.set_zlabel('Real part of $u_1$')

    ax5 = fig.add_subplot(235, projection='3d')
    ax5.plot_surface(X.numpy(), T.numpy(), imag_u1_analytical, cmap=cmap)
    ax5.set_title('Analytical Imaginary Part of $u_1(x, t)$')
    ax5.set_xlabel('x')
    ax5.set_ylabel('t')
    ax5.set_zlabel('Imag part of $u_1$')

    ax6 = fig.add_subplot(236, projection='3d')
    ax6.plot_surface(X.numpy(), T.numpy(), real_v1_analytical, cmap=cmap)
    ax6.set_title('Analytical Real Part of $v_1(x, t)$')
    ax6.set_xlabel('x')
    ax6.set_ylabel('t')
    ax6.set_zlabel('Real part of $v_1$')

    plt.tight_layout()
    plt.savefig(os.path.join(image_save_path, f'chiggs_model_comparison_3d_epoch_{epoch}.png'))
    plt.close(fig)  # Close the figure to free memory


In [8]:
! rm -rf results
! rm -rf model_weights

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available():
    print("CUDA is available! Training on GPU.")
else:
    print("CUDA is not available. Training on CPU.")

model_u = FourierFeatureNN(device=device).to(device)
model_v = FourierFeatureNN_V(device=device).to(device)
#model_u = FCN(input_dim=1, neuron_units=100, activation=nn.Tanh, device=device)
#model_ui = FCN(input_dim=1, neuron_units=100, activation=nn.Tanh, device=device)
#model_v = FCN(input_dim=1, neuron_units=100, activation=nn.Tanh, device=device)

print(model_u)
print(model_v)
num_epochs_lbfgs = 500  # Number of training epochs
num_samples_lbfgs = 1000 # Number of samples for training
num_epochs_sq = 36000
num_samples_sq = 1000
lr_sq = 1e-4 
lr_lbfgs = 1e-2
r = 1.1
omega = 5 
k = 0.5
gamma = 1
beta = 1
model_save_path = 'model_weights' 
mse_cost_function = torch.nn.MSELoss()
os.makedirs(model_save_path, exist_ok=True)
os.makedirs('results', exist_ok=True)
losses = []
line_search_fn = "strong_wolfe"

CUDA is available! Training on GPU.
FourierFeatureNN(
  (path_x): Sequential(
    (0): Linear(in_features=16, out_features=32, bias=True)
    (1): Tanh()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Tanh()
    (4): Linear(in_features=32, out_features=32, bias=True)
    (5): Tanh()
  )
  (path_t): Sequential(
    (0): Linear(in_features=16, out_features=32, bias=True)
    (1): Tanh()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Tanh()
    (4): Linear(in_features=32, out_features=32, bias=True)
    (5): Tanh()
  )
  (ffn_u): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): Tanh()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Tanh()
    (4): Linear(in_features=32, out_features=2, bias=True)
  )
)
FourierFeatureNN_V(
  (path_x): Sequential(
    (0): Linear(in_features=16, out_features=32, bias=True)
    (1): Tanh()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Tanh(

In [ ]:
LBFGS_training(model_u, model_v, model_save_path, mse_cost_function, device, num_epochs_lbfgs, lr_lbfgs, num_samples_lbfgs, r, k, omega, gamma, beta, line_search_fn)

Starting LBFGS Fine Tuning


Progress::   0%|                                                        | ?

tensor(0.0212, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0212, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0213, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0213, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0216, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0216, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0222, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0222, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0250, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0250, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0253, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0253, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0262, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0262, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0339, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0339, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0345, device='cuda:0', grad_fn=

Progress::   0%|                                                  | 5:05:40

tensor(0.5468, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5468, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5455, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5455, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5464, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5464, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5464, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5464, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5428, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5428, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5500, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5500, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5474, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5474, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5474, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5474, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.5427, device='cuda:0', grad_fn=

...

tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', gr

Progress::  23%| ███████████▏                                     | 2:27:39

tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', gr

Progress::  23%| ███████████▏                                     | 2:33:57

tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', gr